# 3 Search services

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import datetime
import folium
import os
import pandas as pd
import requests
import sys

from dotenv import load_dotenv
from IPython.display import IFrame, FileLink
from typing import List, Dict, Tuple, Optional

## Settings

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 01-Sep-2025 08:01:45


In [4]:
load_dotenv("azure.env")

True

In [5]:
RESULTS_DIR = "results"

os.makedirs(RESULTS_DIR, exist_ok=True)

In [6]:
class AzureMapsClient:
    """
    Azure Maps API Client for Python
    """

    def __init__(self, subscription_key: str):
        """
        Initialize Azure Maps client

        Args:
            subscription_key: Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com"

    def _make_request(self, endpoint: str, params: Dict) -> Dict:
        """
        Make authenticated request to Azure Maps API

        Args:
            endpoint: API endpoint
            params: Request parameters

        Returns:
            API response as dictionary
        """
        params['api-version'] = '1.0'
        params['subscription-key'] = self.subscription_key

        url = f"{self.base_url}/{endpoint}"

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return {}

## Helper

In [7]:
class SearchDemo:
    """
    Demonstrates Azure Maps search capabilities
    """

    def __init__(self, azure_maps_client):
        self.client = azure_maps_client

    def fuzzy_search(self, query: str, latitude: float = None, longitude: float = None,
                     radius: int = 10000, limit: int = 10) -> Dict:
        """
        Fuzzy search that handles typos and partial matches

        Args:
            query: Search term
            latitude: Optional center point latitude
            longitude: Optional center point longitude
            radius: Search radius in meters
            limit: Maximum number of results

        Returns:
            Search results
        """
        params = {
            'query': query,
            'limit': limit
        }

        if latitude and longitude:
            params['lat'] = latitude
            params['lon'] = longitude
            params['radius'] = radius

        response = self.client._make_request('search/fuzzy/json', params)
        return response

    def category_search(self, category: str, latitude: float, longitude: float,
                        radius: int = 10000, limit: int = 20) -> Dict:
        """
        Search for places by category

        Args:
            category: POI category (e.g., "restaurant", "gas station", "hospital")
            latitude: Center point latitude
            longitude: Center point longitude
            radius: Search radius in meters
            limit: Maximum number of results

        Returns:
            Category search results
        """
        params = {
            'query': category,
            'lat': latitude,
            'lon': longitude,
            'radius': radius,
            'limit': limit
        }

        response = self.client._make_request(
            'search/poi/category/json', params)
        return response

    def poi_search(self, query: str, latitude: float = None, longitude: float = None,
                   radius: int = 10000, limit: int = 10) -> Dict:
        """
        Search for specific points of interest

        Args:
            query: POI search term
            latitude: Optional center point latitude
            longitude: Optional center point longitude
            radius: Search radius in meters
            limit: Maximum number of results

        Returns:
            POI search results
        """
        params = {
            'query': query,
            'limit': limit
        }

        if latitude and longitude:
            params['lat'] = latitude
            params['lon'] = longitude
            params['radius'] = radius

        response = self.client._make_request('search/poi/json', params)
        return response

    def nearby_search(self, latitude: float, longitude: float,
                      radius: int = 5000, limit: int = 20) -> Dict:
        """
        Find nearby points of interest

        Args:
            latitude: Center point latitude
            longitude: Center point longitude
            radius: Search radius in meters
            limit: Maximum number of results

        Returns:
            Nearby POI results
        """
        params = {
            'lat': latitude,
            'lon': longitude,
            'radius': radius,
            'limit': limit
        }

        response = self.client._make_request('search/nearby/json', params)
        return response

In [8]:
def extract_coordinates(result: Dict) -> Tuple[float, float]:
    """
    Extract latitude and longitude from Azure Maps result
    
    Args:
        result: Azure Maps API result object
        
    Returns:
        Tuple of (latitude, longitude)
    """
    if 'position' in result:
        pos = result['position']
        return pos['lat'], pos['lon']
    return None, None

## Examples

In [9]:
# Initialize search demo
search = SearchDemo(AzureMapsClient(os.getenv('AZURE_MAPS_KEY')))

### Example 1: Fuzzy Search

In [10]:
# Search with intentional typos
fuzzy_queries = ["starbcks", "mcdonalds", "walmart", "hospitl"]

for query in fuzzy_queries:
    print(f"\nSearching for '{query}':")
    result = search.fuzzy_search(query, limit=3)

    if result and 'results' in result:
        for i, res in enumerate(result['results']):
            name = res.get('poi', {}).get('name', 'N/A')
            category = res.get('poi',
                               {}).get('categories', ['N/A'])[0] if res.get(
                                   'poi', {}).get('categories') else 'N/A'
            address = res.get('address', {}).get('freeformAddress', 'N/A')
            distance = res.get('dist', 'N/A')

            print(f"  {i+1}. {name} ({category})")
            print(f"     Address: {address}")
            
            if distance != 'N/A':
                print(f"     Distance: {distance:.0f}m")


Searching for 'starbcks':
  1. N/A (N/A)
     Address: Starbäcksvägen, 73532, 73537 Surahammar
  2. Mural Tatería Starbucks (important tourist attraction)
     Address: Boulevard Primavera 3028, 64830, Monterrey, Nuevo León
  3. N/A (N/A)
     Address: Staricks Road, Murtoa, VIC, 3390

Searching for 'mcdonalds':
  1. N/A (N/A)
     Address: McDonalds Corners ON
  2. McDonald's (important tourist attraction)
     Address: 9 Church Street, Leatherhead, KT22 8DN
  3. N/A (N/A)
     Address: McDonalds Road, South Coast, WA

Searching for 'walmart':
  1. N/A (N/A)
     Address: Walmart, Nipsan Yahukimo, Papua Pegunungan
  2. N/A (N/A)
     Address: Walmart Drive, Eatonton, GA 31024
  3. N/A (N/A)
     Address: Walmart Court, Eastanollee, GA 30538

Searching for 'hospitl':
  1. Motto Hospitl (hotel/motel)
     Address: Gang Astana, Cihideung Sub District, Kota Tasikmalaya District, Jawa Barat 46126
  2. Hospitl Miftkhul Huda (hotel)
     Address: Mesuji Makmur Sub District, Ogan Komering Di

## Example 2: Category Search around a specific location

In [11]:
lat, lon = 47.643543, -122.130821

restaurant_results = search.category_search(
    category="restaurant",
    latitude=lat,
    longitude=lon,
    radius=1000,  # 3km radius
    limit=20)

restaurant_data = []
if restaurant_results and 'results' in restaurant_results:
    print(f"Found {len(restaurant_results['results'])} restaurants:")

    for i, res in enumerate(restaurant_results['results']):
        name = res.get('poi', {}).get('name', 'N/A')
        categories = res.get('poi', {}).get('categories', [])
        address = res.get('address', {}).get('freeformAddress', 'N/A')
        lat, lon = extract_coordinates(res)
        distance = res.get('dist', 'N/A')

        print(f"  {i+1}. {name}")
        print(
            f"     Categories: {', '.join(categories) if categories else 'N/A'}"
        )
        print(f"     Address: {address}")
        print(f"     Distance: {distance:.0f}m"
              if distance != 'N/A' else "     Distance: N/A")
        print()

        restaurant_data.append({
            'name':
            name,
            'categories':
            ', '.join(categories) if categories else 'N/A',
            'address':
            address,
            'latitude':
            lat,
            'longitude':
            lon,
            'distance':
            distance
        })

Found 20 restaurants:
  1. Microsoft Cafe 16
     Categories: restaurant
     Address: 15701 Northeast 39Th Street, Redmond, WA 98052
     Distance: 174m

  2. Salt & Straw @ Microsoft Building 6
     Categories: ice cream parlor, restaurant
     Address: 15885 Northeast 36th Street, Redmond, WA 98052
     Distance: 190m

  3. Microsoft Cafe 25
     Categories: fast food, restaurant
     Address: 15700 Northeast 39th Street, Redmond, WA 98052
     Distance: 205m

  4. Craft75 Gastropub
     Categories: restaurant
     Address: Redmond, WA 98007
     Distance: 227m

  5. Microsoft Food Hall 4
     Categories: fast food, restaurant
     Address: 3400 156th Avenue Northeast, Redmond, WA 98052
     Distance: 335m

  6. Microsoft Cafe 43
     Categories: fast food, restaurant
     Address: Northeast 33rd Street, Redmond, WA 98007
     Distance: 433m

  7. In.Gredients
     Categories: restaurant
     Address: 3720 159th Avenue Northeast, Redmond, WA 98052
     Distance: 494m

  8. Pike Plac

### Example 3: Multiple Category Search

In [12]:
# Seattle coordinates
seattle_lat, seattle_lon = 47.6062, -122.3321

categories_to_search = [("hospital", "🏥"), ("gas station", "⛽"),
                        ("pharmacy", "💊"), ("bank", "🏦")]

all_pois = []

for category, emoji in categories_to_search:
    print(f"\n{emoji} Searching for {category}s in Seattle:")

    results = search.category_search(category=category,
                                     latitude=seattle_lat,
                                     longitude=seattle_lon,
                                     radius=5000,
                                     limit=5)

    if results and 'results' in results:
        for i, res in enumerate(results['results']):
            name = res.get('poi', {}).get('name', 'N/A')
            address = res.get('address', {}).get('freeformAddress', 'N/A')
            lat, lon = extract_coordinates(res)
            distance = res.get('dist', 'N/A')

            print(f"  {i+1}. {name}")
            print(f"     {address}")
            print(f"     Distance: {distance:.0f}m"
                  if distance != 'N/A' else "     Distance: N/A")

            all_pois.append({
                'name': name,
                'category': category,
                'emoji': emoji,
                'address': address,
                'latitude': lat,
                'longitude': lon,
                'distance': distance
            })


🏥 Searching for hospitals in Seattle:
  1. The Polyclinic Madison Center
     904 7Th Ave, Seattle, WA 98104
     Distance: 255m
  2. Virginia Mason Medical Center
     1100 9Th Ave, Seattle, WA 98101
     Distance: 557m
  3. Harborview
     8Th Avenue, Seattle, WA 98104
     Distance: 617m
  4. Trauma Emergency Ent
     499 9th Avenue, Seattle, WA 98104
     Distance: 619m
  5. Swedish Orthopedic Institute
     801 Alaskan Way, Seattle, WA 98104
     Distance: 634m

⛽ Searching for gas stations in Seattle:
  1. 76
     914 James Street, Seattle, WA 98104
     Distance: 600m
  2. Chevron
     427 12Th Avenue, Seattle, WA 98122
     Distance: 1116m
  3. Shell
     700 12th Avenue, Seattle, WA 98122
     Distance: 1192m
  4. Shell
     511 South Dearborn Street, Seattle, WA 98134
     Distance: 1237m
  5. 76
     1500 Broadway, Seattle, WA 98122
     Distance: 1252m

💊 Searching for pharmacys in Seattle:
  1. Miyuki Anderson
     910 4th Avenue, Seattle, WA 98164
     Distance: 19m
  2.

### Example 4: POI Search for specific chains

In [13]:
# Search for specific coffee chains in San Francisco
sf_lat, sf_lon = 37.7749, -122.4194

coffee_chains = ["Starbucks", "Dunkin", "Peet's Coffee"]

coffee_data = []
for chain in coffee_chains:
    print(f"\nSearching for {chain} locations in San Francisco:")

    results = search.poi_search(query=chain,
                                latitude=sf_lat,
                                longitude=sf_lon,
                                radius=3000,
                                limit=5)

    if results and 'results' in results:
        for i, res in enumerate(results['results']):
            name = res.get('poi', {}).get('name', 'N/A')
            address = res.get('address', {}).get('freeformAddress', 'N/A')
            lat, lon = extract_coordinates(res)
            distance = res.get('dist', 'N/A')

            print(f"  {i+1}. {name}")
            print(f"     {address}")
            print(f"     Distance: {distance:.0f}m"
                  if distance != 'N/A' else "     Distance: N/A")

            coffee_data.append({
                'chain': chain,
                'name': name,
                'address': address,
                'latitude': lat,
                'longitude': lon,
                'distance': distance
            })


Searching for Starbucks locations in San Francisco:
  1. Starbucks
     2675 Geary Boulevard, San Francisco, CA 94118
     Distance: 2560m
  2. Starbucks
     201 Powell Street, San Francisco, CA 94102
     Distance: 1628m
  3. Starbucks
     350 Rhode Island Street, San Francisco, CA 94103
     Distance: 1722m
  4. Starbucks
     2020 Market Street, San Francisco, CA 94114
     Distance: 1022m
  5. Starbucks
     2300 16th Street, San Francisco, CA 94103
     Distance: 1242m

Searching for Dunkin locations in San Francisco:
  1. Isadora Duncan Plaque
     501 Taylor Street, San Francisco, CA 94102
     Distance: 1519m
  2. Douglas Durkin Design
     39 Arkansas Street, San Francisco, CA 94107
     Distance: 2102m

Searching for Peet's Coffee locations in San Francisco:
  1. Peet's Coffee
     1400 Mission Street, San Francisco, CA 94103
     Distance: 285m
  2. Peet's Coffee
     555 9th Street, San Francisco, CA 94103
     Distance: 1143m
  3. Peet's Coffee
     2300 16th Street, Sa

### Example 5: Nearby Search

In [14]:
# Central Park coordinates
central_park_lat, central_park_lon = 40.7829, -73.9654

nearby_results = search.nearby_search(
    latitude=central_park_lat,
    longitude=central_park_lon,
    radius=500,  # 500m radius
    limit=10)

if nearby_results and 'results' in nearby_results:
    print(f"Found {len(nearby_results['results'])} places near Central Park:")

    for i, res in enumerate(nearby_results['results']):
        name = res.get('poi', {}).get('name', 'N/A')
        categories = res.get('poi', {}).get('categories', [])
        address = res.get('address', {}).get('freeformAddress', 'N/A')
        distance = res.get('dist', 'N/A')

        print(f"  {i+1}. {name}")
        print(
            f"     Categories: {', '.join(categories[:2]) if categories else 'N/A'}"
        )
        print(f"     Address: {address}")
        print(f"     Distance: {distance:.0f}m"
              if distance != 'N/A' else "     Distance: N/A")
        print()

Found 10 places near Central Park:
  1. Central Park Tours
     Categories: company
     Address: New York, NY 10024
     Distance: 6m

  2. Central Park Airlines - Pedicab
     Categories: company
     Address: Central Park West, New York, NY 10024
     Distance: 37m

  3. Marka Reklam
     Categories: advertising/marketing, company
     Address: 50 85Th Street Transverse, New York, NY 10024
     Distance: 55m

  4. Treebath
     Categories: company
     Address: Central Park West, New York, NY 10024
     Distance: 62m

  5. Central Park
     Categories: park, park recreation area
     Address: West Drive, New York, NY 10024
     Distance: 62m

  6. Great Lawn Basketball Courts
     Categories: sports center
     Address: Great Lawn Oval, New York, NY 10024
     Distance: 95m

  7. West-Park Korean Church
     Categories: place of worship
     Address: 165 West 86th Transverse Road, New York, NY 10024
     Distance: 107m

  8. Mesilah
     Categories: place of worship
     Address: 13

### Example 6: Create comprehensive search map

In [15]:
def create_search_map(poi_data: List[Dict],
                      center_coords: Tuple[float, float],
                      map_title: str = "Search Results") -> folium.Map:
    """
    Create folium map with search results
    
    Args:
        poi_data: List of POI dictionaries
        center_coords: Center coordinates for map
        map_title: Title for the map
        
    Returns:
        Folium map object
    """
    center_lat, center_lon = center_coords

    # Create map
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=12,
                   tiles='OpenStreetMap')

    # Color mapping for different categories
    color_map = {
        'restaurant': 'red',
        'hospital': 'green',
        'gas station': 'blue',
        'pharmacy': 'purple',
        'bank': 'orange',
        'coffee': 'darkred'
    }

    # Add markers for each POI
    for poi in poi_data:
        category = poi.get('category', 'unknown')
        emoji = poi.get('emoji', '📍')
        color = color_map.get(category, 'gray')

        popup_text = f"<b>{poi['name']}</b><br>"
        popup_text += f"Category: {category}<br>"
        popup_text += f"Address: {poi['address']}<br>"
        if poi.get('distance') != 'N/A':
            popup_text += f"Distance: {poi['distance']:.0f}m"

        folium.Marker(location=[poi['latitude'], poi['longitude']],
                      popup=popup_text,
                      tooltip=f"{emoji} {poi['name']}",
                      icon=folium.Icon(color=color,
                                       icon='info-sign')).add_to(m)

    # Add center point
    folium.Marker(location=[center_lat, center_lon],
                  popup="<b>Search Center</b>",
                  tooltip="Search Center",
                  icon=folium.Icon(color='black', icon='star')).add_to(m)

    return m


# Create map with Seattle POI data
if all_pois:
    seattle_search_map = create_search_map(all_pois,
                                           (seattle_lat, seattle_lon),
                                           "Seattle POI Search Results")
    output_file = os.path.join(RESULTS_DIR, "azure_maps_seattle_search.html")
    seattle_search_map.save(output_file)
    print(f"✅ Map saved as {output_file}")

# Create DataFrame summary
print("\nSearch Results Summary")
print("-" * 26)

if all_pois:
    df_search = pd.DataFrame(all_pois)
    category_counts = df_search['category'].value_counts()

    print("POI Categories Found:")
    for category, count in category_counts.items():
        emoji = df_search[df_search['category'] == category]['emoji'].iloc[0]
        print(f"  {emoji} {category.title()}: {count} locations")

    print(f"\nTotal POIs found: {len(all_pois)}")

✅ Map saved as results/azure_maps_seattle_search.html

Search Results Summary
--------------------------
POI Categories Found:
  🏥 Hospital: 5 locations
  ⛽ Gas Station: 5 locations
  💊 Pharmacy: 5 locations
  🏦 Bank: 5 locations

Total POIs found: 20


In [16]:
IFrame(src=output_file, width=800, height=600)

In [17]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/azure_maps_seattle_search.html